In [ ]:
import sqlite3
import pathlib
import logging
import datetime
#from pandas import DataFrame, read_csv
import pandas as pd
from llama_index.core.utilities.sql_wrapper import SQLDatabase
from llama_index.core.indices.struct_store import (
    NLSQLTableQueryEngine,
    SQLTableRetrieverQueryEngine,
)
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Numeric, Date, ForeignKey
from sqlalchemy import insert
import os
import openai
import re
os.environ["DC_STATEHOOD"] = "1"
import us
# code: https://github.com/xie1027/GenAI_Insights/blob/main/examples/examples_acs_g4.py

In [ ]:
#data_key = 'acs_dem.csv' 
#data_location = 's3://{}/{}'.format(bucket, data_key)

acs_df_dem = pd.read_csv('/home/sagemaker-user/acs_dem.csv')

bucket = 'insight-query-bucket'
data_key = 'acs_econ.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
acs_df_econ = pd.read_csv(data_location)

data_key = 'acs_housing.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
acs_df_housing = pd.read_csv(data_location)

Populate Tables in Database (skip if the database is populated)

In [ ]:
engine = create_engine("sqlite:///acs.sqlite", future=True)
conn = engine.connect()
metadata = MetaData()

In [ ]:
table_name = "acs_dem"
acs_dem_table = Table(
    table_name,
    metadata,
)
for i, column_name in enumerate(acs_df_dem.columns):
    if i == 0:  # First column is a String
        acs_dem_table.append_column(Column(column_name, String(20), primary_key=True))
    elif i == 1:  # Second column is Numeric and marked as a primary key
        acs_dem_table.append_column(Column(column_name, Numeric, primary_key=True))
    else:  # The rest of the columns are Numeric and nullable
        acs_dem_table.append_column(Column(column_name, Numeric, nullable=True))

table_name = "acs_econ"
acs_econ_table = Table(
    table_name,
    metadata,
)
for i, column_name in enumerate(acs_df_econ.columns):
    if i == 0:  # First column is a String
        acs_econ_table.append_column(Column(column_name, String(20), primary_key=True))
    elif i == 1:  # Second column is Numeric and marked as a primary key
        acs_econ_table.append_column(Column(column_name, Numeric, primary_key=True))
    else:  # The rest of the columns are Numeric and nullable
        acs_econ_table.append_column(Column(column_name, Numeric, nullable=True))

table_name = "acs_housing"
acs_housing_table = Table(
    table_name,
    metadata,
)
for i, column_name in enumerate(acs_df_housing.columns):
    if i == 0:  # First column is a String
        acs_housing_table.append_column(Column(column_name, String(20), primary_key=True))
    elif i == 1:  # Second column is Numeric and marked as a primary key
        acs_housing_table.append_column(Column(column_name, Numeric, primary_key=True))
    else:  # The rest of the columns are Numeric and nullable
        acs_housing_table.append_column(Column(column_name, Numeric, nullable=True))

In [ ]:
metadata.create_all(engine)

In [ ]:
# Insert rows
# Each dictionary represents a row in the DataFrame, with column names as keys
rows_to_insert = acs_df_dem.to_dict(orient='records')
# Connect to the database
for row in rows_to_insert:
    stmt = insert(acs_dem_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

rows_to_insert = acs_df_econ.to_dict(orient='records')
for row in rows_to_insert:
    stmt = insert(acs_econ_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

rows_to_insert = acs_df_housing.to_dict(orient='records')
for row in rows_to_insert:
    stmt = insert(acs_housing_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()